In [57]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier

from sklearn.pipeline import Pipeline

In [7]:
# set random seed
np.random.seed(1294)

In [2]:
train = pd.read_csv('../data/train.csv', index_col='ID')
test = pd.read_csv('../data/test.csv', index_col='ID')

In [4]:
features = train.columns.drop('TARGET')

X = train[features]
y = train.TARGET

In [6]:
# number of zeros in each row as feature
X['num_zeros'] = (X == 0).astype(int).sum(axis=1)
test['num_zeros'] = (test == 0).astype(int).sum(axis=1)

In [15]:
X['num_zeros_greater_than_median'] = X.num_zeros.map(lambda x: int(x > 340))
test['num_zeros_greater_than_median'] = test.num_zeros.map(lambda x: int(x > 340))

## Train and Test Split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=44)

In [67]:
scaler = StandardScaler()
pca = PCA(n_components=100, whiten=True)
clf = XGBClassifier(learning_rate=0.1, min_child_weight=5, scale_pos_weight=5, seed=1234, \
                    n_estimators=100, colsample_bytree=0.7, subsample=0.6)

pipeline = Pipeline([('scaler', scaler), ('pca', pca), ('clf', clf)])

In [68]:
# fit on the training examples
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, n_components=100, whiten=True)), ('clf', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=5, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=5, seed=1234, silent=True, subsample=0.6))])

In [69]:
# predictions on the training and test set
print 'ROC AUC score on training examples : %.3f ' %(roc_auc_score(y_train, pipeline.predict_proba(X_train)[:, 1]))
print 'ROC AUC score on unseen examples : %.3f ' %(roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1]))

ROC AUC score on training examples : 0.880 
ROC AUC score on unseen examples : 0.813 


## Train on full dataset

In [70]:
pipeline.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, n_components=100, whiten=True)), ('clf', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=5, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=5, seed=1234, silent=True, subsample=0.6))])

## Prediction

In [71]:
predictions = pipeline.predict_proba(test)[:, 1]

## Submission

In [72]:
submission = pd.read_csv('../data/sample_submission.csv')
submission['TARGET'] = predictions
submission.to_csv('../submissions/xgb_pipeline', index=False)